In [1]:
# Add lib input sys.path
import os
import sys
import time

import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import math
from sklearn.preprocessing import normalize
from functools import partial
import h5py
from scipy.spatial import distance

nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

from matplotlib.colors import ListedColormap
import libs.linear_models as lm
import libs.data_util as data
import libs.nn as nn
import libs.plot as myplot
from sklearn import decomposition
from sklearn.model_selection import train_test_split
from scipy.optimize import minimize_scalar

%matplotlib inline

#### Exercise B.1

* (a) 
  * There is only one vector in the first set, it is independent.
  * The vector $\begin{bmatrix}1\\2\end{bmatrix}$ and $\begin{bmatrix}2\\4\end{bmatrix}$ are dependent.
  * The vector $\begin{bmatrix}1\\2\end{bmatrix}$ and $\begin{bmatrix}1\\1\end{bmatrix}$ are independent.
  * since $-2\begin{bmatrix}1\\2\end{bmatrix} + 5 \begin{bmatrix}1\\1\end{bmatrix} = \begin{bmatrix}3\\1\end{bmatrix}$, the vectors are dependent.

* (b) Both the vectors in set 3 and set 4 can span $R^2$.

* (c) If $v_1,\dots, v_d$ are independent, suppose $u=\sum^d_{i=1}\alpha_i v_i$, if $\alpha_i$ are not unique, i.e. there exists $\beta \ne \alpha$ such that $u=\sum^d_{i=1}\beta_iv_i$. Then we have $\sum^d_{i=1}(\alpha_i - \beta_i)v_i = 0$, since $\beta \ne \alpha$, we have a non-trivial linear combination of $v_i$ that equals to 0. This contradicts with the property that $v_i$ are independent. So $\alpha$ is uniuqe.

Suppose $u=\sum^d_{i=1}\alpha_i v_i$ holds for unique $\alpha_i$, if $v_i$ are not independent, then we have non-trivial solution to $\sum^d_{i=1}\beta_i v_i = 0$, we can write, e.g. $v_d = \sum^{d-1}_{i=1}\gamma_i v_i$, take this into $u$, we have $u=\sum^d_{i=1} \delta_i v_i$ where $\delta_d = 0$. This contradicts with the assumption that $\alpha$ is uniuqe. So $v_i$ must be independent.

* (d) The vectors in set 3 are a basis for $R^2$.
* (e) 